In [1]:
!pip install -q transformers accelerate bitsandbytes sentencepiece
!pip install -U bitsandbytes

In [5]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ──────────────── 1️⃣  CHOOSE A MODEL ───────────────── #
# MODEL_ID = "emilyalsentzer/Bio_Discharge_Summary_BERT"      # Discharge-BERT
# MODEL_ID = "epfl-llm/meditron-7b"                           # Meditron-7B
# MODEL_ID = "microsoft/biogpt"                               # BioGPT (decoder-only)
# MODEL_ID = "mradermacher/JSL-Med-Mistral-24B-V1-Slerp-i1-GGUF"   # JSL-Mistral-24B (GGUF)
# MODEL_ID = "mradermacher/JSL-MedQwen-14b-reasoning-i1-GGUF"      # JSL-Qwen-14B (GGUF)
# MODEL_ID = "mradermacher/JSL-Med-Phi-3.5-Mini-v3-i1-GGUF"        # JSL-Phi-3.5 (GGUF)
# MODEL_ID = "mradermacher/Llama-3.1-8B-UltraMedical-i1-GGUF"      # Llama-UltraMedical-8B
MODEL_ID = "epfl-llm/meditron-7b"                               # ← pick one

# ──────────────── 2️⃣  PROMPT (keep as-is) ───────────────── #
prompt = """You are an experienced clinical pharmacist so first extract medicine details only from below prompt.
For every medicine (one per line) in the list below, classify it as
RELEVANT for treating or managing "type-2 diabetes" or IRRELEVANT
(unrelated/contraindicated). Return strictly valid JSON:
{
  "relevant": [
    {"name":"<Med A>","explanation":"<15-30 word reason it helps type-2 diabetes>"}
  ],
  "irrelevant": [
    {"name":"<Med X>","explanation":"<brief reason it is not used / risky>"}
  ]
}
• Keep keys exactly as shown.
• Do not add any explanations outside the JSON block.
List:
Metformin
Aspirin
Lisinopril
Hydrochlorothiazide
Ibuprofen
Atorvastatin
Paracetamol
Omeprazole
Vitamin D
Cetirizine
"""

# ──────────────── 3️⃣  HYPER-PARAMETERS ───────────────── #
MAX_NEW_TOKENS = 256
TEMPERATURE    = 0.2
TOP_P          = 0.8
DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE          = torch.float16 if DEVICE == "cuda" else torch.float32

# ──────────────── 4️⃣  LOAD MODEL ───────────────── #
# For 7B-13B models, full-precision may fit on a single modern GPU;
# for ≥24B or laptop use, enable 4-bit quantisation.
quant_cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=DTYPE)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=DTYPE,
    quantization_config=quant_cfg
)

# ──────────────── 5️⃣  INFERENCE ───────────────── #
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

with torch.no_grad():
    gen_ids = model.generate(
        **inputs,
        max_new_tokens = MAX_NEW_TOKENS,
        do_sample      = True,
        temperature    = TEMPERATURE,
        top_p          = TOP_P,
        pad_token_id   = tokenizer.eos_token_id
    )

response = tokenizer.decode(
    gen_ids[0][inputs["input_ids"].shape[-1]:],
    skip_special_tokens=True
).strip()

print("🔹 GENERATED JSON")
print(response)


tokenizer_config.json:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]